In [1]:
import pandas as pd 
#%pip install langchain_groq
from langchain_groq.chat_models import ChatGroq
import pandas as pd
import numpy as np
from itertools import product
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score
import ast
import time
import matplotlib.pyplot as plt
import tsfel
import os

In [28]:
# Groq API and Models 
Groq_Token = open('..\\groqapi.txt', 'r').read().split("\n")[0]  # Do not share this key with anyone
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

Let us load the collected data

In [3]:
x_test = []
y_test = []
activity_dic = {1: "WALKING", 2: "WALKING_UPSTAIRS", 3: "WALKING_DOWNSTAIRS", 4: "SITTING", 5: "STANDING", 6: "LAYING"}

In [4]:
for activity in os.listdir("ProcessedData"):
    for file in os.listdir("ProcessedData\\"+activity):
        data = pd.read_csv("ProcessedData\\"+activity+"\\"+file)
        x_test.append(list(data.values))
        y_test.append(activity)
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_test.shape)
print(y_test.shape)

(24, 500, 3)
(24,)


Let us collect the UCI data

In [5]:
x_train = np.load("..\\x_train.npy")
y_train = np.load("..\\y_train.npy")

In [6]:
cfg = tsfel.get_features_by_domain()
feature_df = tsfel.time_series_features_extractor(cfg, x_train, verbose = 0, fs = 50)
feature_df_test = tsfel.time_series_features_extractor(cfg, x_test, verbose = 0, fs = 50)

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
scaler = StandardScaler()
X_scaled = scaler.fit_transform(feature_df)
# Select top 20 features
selector = SelectKBest(score_func=f_classif, k=20)
X_reduced = selector.fit_transform(X_scaled, y_train)
selected_feature_indices = selector.get_support(indices=True)
selected_feature_names = feature_df.columns[selected_feature_indices]
print("Selected features:", selected_feature_names)

Selected features: Index(['0_Absolute energy', '0_Area under the curve', '0_Average power',
       '0_ECDF Percentile_0', '0_ECDF Percentile_1', '0_Max', '0_Mean',
       '0_Median', '0_Peak to peak distance', '0_Root mean square',
       '0_Spectral distance', '0_Standard deviation',
       '0_Wavelet absolute mean_7', '0_Wavelet absolute mean_8',
       '0_Wavelet energy_6', '0_Wavelet energy_7', '0_Wavelet energy_8',
       '0_Wavelet standard deviation_6', '0_Wavelet standard deviation_7',
       '0_Wavelet standard deviation_8'],
      dtype='object')


c:\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  5   6   9  10  11  12  13  14  15  16  17  18 272 273 274 275 338 389
 390 393 394 395 396 397 398 399 400 401 402 773 774 777 778 779 780 781
 782 783 784 785 786] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [8]:
x_train_reduced = feature_df[selected_feature_names]
x_test_reduced = feature_df_test[selected_feature_names]
display(x_train_reduced); display(x_test_reduced)

,0_Absolute energy,0_Area under the curve,0_Average power,0_ECDF Percentile_0,0_ECDF Percentile_1,0_Max,0_Mean,0_Median,0_Peak to peak distance,0_Root mean square,0_Spectral distance,0_Standard deviation,0_Wavelet absolute mean_7,0_Wavelet absolute mean_8,0_Wavelet energy_6,0_Wavelet energy_7,0_Wavelet energy_8,0_Wavelet standard deviation_6,0_Wavelet standard deviation_7,0_Wavelet standard deviation_8
0,428.119592,9.008919,42.897755,0.737727,1.086936,1.514191,0.902649,0.875090,1.016695,0.925332,-93529.983215,0.203626,0.070754,0.084397,0.748413,0.781690,0.767415,0.746177,0.778481,0.762760
1,32.768542,2.553859,3.283421,0.249165,0.262091,0.275936,0.255891,0.255506,0.035738,0.256002,-17287.032136,0.007540,0.020233,0.024193,0.064288,0.073404,0.082654,0.062128,0.070561,0.079035
2,478.627177,9.764364,47.958635,0.975656,0.980947,0.991023,0.978388,0.978214,0.023335,0.978394,-62134.689305,0.003478,0.076864,0.091693,0.251557,0.287185,0.322695,0.243559,0.276708,0.309393
3,477.002571,9.495337,47.795849,0.762545,1.075617,1.792962,0.951163,0.903567,1.188075,0.976732,-97287.429440,0.222022,0.080304,0.095454,0.811481,0.879340,0.899850,0.808800,0.875666,0.894773
4,530.466450,10.005161,53.152951,0.770530,1.234635,1.586526,1.001998,0.957724,0.955737,1.030016,-100789.226316,0.238608,0.077136,0.095418,0.924487,0.970024,0.956793,0.922525,0.966952,0.952023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,497.739114,9.755190,49.873659,0.784800,1.202131,1.368604,0.977764,0.954234,0.856041,0.997737,-85649.703487,0.198636,0.075838,0.090382,0.671130,0.651775,0.599676,0.668243,0.647348,0.592826
122,340.897228,8.207162,34.158039,0.776342,0.926267,1.069117,0.822427,0.779678,0.373051,0.825708,-62295.662275,0.073545,0.066999,0.079946,0.233461,0.266846,0.299092,0.226930,0.258298,0.288209
123,17.727040,1.878521,1.776256,0.183463,0.192260,0.199627,0.188234,0.188654,0.021505,0.188293,-12633.064233,0.004684,0.015061,0.017969,0.049603,0.056622,0.063587,0.048048,0.054583,0.060996
124,430.187347,9.028684,43.104945,0.732451,1.062091,1.624140,0.904258,0.848261,1.119199,0.927564,-95272.557083,0.206619,0.066492,0.082922,0.708120,0.761477,0.778275,0.706229,0.758569,0.773845


,0_Absolute energy,0_Area under the curve,0_Average power,0_ECDF Percentile_0,0_ECDF Percentile_1,0_Max,0_Mean,0_Median,0_Peak to peak distance,0_Root mean square,0_Spectral distance,0_Standard deviation,0_Wavelet absolute mean_7,0_Wavelet absolute mean_8,0_Wavelet energy_6,0_Wavelet energy_7,0_Wavelet energy_8,0_Wavelet standard deviation_6,0_Wavelet standard deviation_7,0_Wavelet standard deviation_8
0,15.877739,1.269697,1.590956,0.073849,0.115618,0.840807,0.116540,0.091932,1.193411,0.178201,-32466.639193,0.134810,0.028233,0.033021,0.198406,0.222017,0.249537,0.197005,0.220215,0.247343
1,6.821886,1.160060,0.683556,0.107375,0.124901,0.169396,0.116288,0.115496,0.083123,0.116807,-8701.944950,0.010994,0.009816,0.011631,0.035688,0.039755,0.043774,0.034756,0.038525,0.042200
2,4.153816,0.821470,0.416214,0.046852,0.118585,0.252132,0.082228,0.068772,0.241005,0.091146,-10506.476634,0.039321,0.005818,0.007109,0.030824,0.032070,0.035189,0.030460,0.031538,0.034463
3,0.850436,0.409766,0.085214,-0.044469,-0.037811,-0.029742,-0.041065,-0.041150,0.021440,0.041242,-3177.982568,0.003812,0.003139,0.003735,0.010845,0.012168,0.013521,0.010534,0.011756,0.012995
4,412.251192,9.016672,41.307735,0.879704,0.922294,1.560766,0.904129,0.899124,0.953330,0.908021,-74127.769363,0.083986,0.071202,0.085061,0.269501,0.307157,0.339420,0.263170,0.298790,0.328589
5,444.294787,9.407062,44.518516,0.933025,0.951843,0.979106,0.942571,0.943104,0.080884,0.942650,-60408.079633,0.012218,0.073620,0.087918,0.237663,0.271994,0.306509,0.229911,0.261841,0.293630
6,471.512038,9.691130,47.245695,0.962886,0.979063,1.010948,0.971039,0.970738,0.076751,0.971094,-61734.552793,0.010386,0.076115,0.090869,0.246947,0.282361,0.317724,0.238969,0.271908,0.304452
7,245.836137,6.997621,24.632879,0.697045,0.705353,0.724764,0.701173,0.701127,0.037039,0.701193,-44729.957205,0.005317,0.054997,0.065618,0.179579,0.205128,0.230764,0.173844,0.197618,0.221238
8,458.788306,9.558291,45.970772,0.948995,0.967920,1.025856,0.957765,0.958687,0.148498,0.957902,-61289.725185,0.016222,0.075423,0.089998,0.246451,0.281772,0.317003,0.238595,0.271491,0.303959
9,448.472751,9.450841,44.937149,0.937170,0.956961,0.981736,0.946993,0.946969,0.087129,0.947072,-60821.911866,0.012225,0.074818,0.089254,0.245764,0.280771,0.315532,0.238011,0.270620,0.302645


In [9]:
x_train = np.load("..\\X_train.npy")
y_train = np.load("..\\Y_train.npy")
y_train = np.array([activity_dic[i] for i in y_train])
x_train_reduced['Activity'] = y_train
x_test_reduced['Activity'] = y_test

C:\Users\Rajeev Wankar\AppData\Local\Temp\ipykernel_18532\1343023622.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train_reduced['Activity'] = y_train
C:\Users\Rajeev Wankar\AppData\Local\Temp\ipykernel_18532\1343023622.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_reduced['Activity'] = y_test


In [10]:
display(x_train_reduced); display(x_test_reduced)

,0_Absolute energy,0_Area under the curve,0_Average power,0_ECDF Percentile_0,0_ECDF Percentile_1,0_Max,0_Mean,0_Median,0_Peak to peak distance,0_Root mean square,...,0_Standard deviation,0_Wavelet absolute mean_7,0_Wavelet absolute mean_8,0_Wavelet energy_6,0_Wavelet energy_7,0_Wavelet energy_8,0_Wavelet standard deviation_6,0_Wavelet standard deviation_7,0_Wavelet standard deviation_8,Activity
0,428.119592,9.008919,42.897755,0.737727,1.086936,1.514191,0.902649,0.875090,1.016695,0.925332,...,0.203626,0.070754,0.084397,0.748413,0.781690,0.767415,0.746177,0.778481,0.762760,WALKING_UPSTAIRS
1,32.768542,2.553859,3.283421,0.249165,0.262091,0.275936,0.255891,0.255506,0.035738,0.256002,...,0.007540,0.020233,0.024193,0.064288,0.073404,0.082654,0.062128,0.070561,0.079035,LAYING
2,478.627177,9.764364,47.958635,0.975656,0.980947,0.991023,0.978388,0.978214,0.023335,0.978394,...,0.003478,0.076864,0.091693,0.251557,0.287185,0.322695,0.243559,0.276708,0.309393,STANDING
3,477.002571,9.495337,47.795849,0.762545,1.075617,1.792962,0.951163,0.903567,1.188075,0.976732,...,0.222022,0.080304,0.095454,0.811481,0.879340,0.899850,0.808800,0.875666,0.894773,WALKING_UPSTAIRS
4,530.466450,10.005161,53.152951,0.770530,1.234635,1.586526,1.001998,0.957724,0.955737,1.030016,...,0.238608,0.077136,0.095418,0.924487,0.970024,0.956793,0.922525,0.966952,0.952023,WALKING_UPSTAIRS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,497.739114,9.755190,49.873659,0.784800,1.202131,1.368604,0.977764,0.954234,0.856041,0.997737,...,0.198636,0.075838,0.090382,0.671130,0.651775,0.599676,0.668243,0.647348,0.592826,WALKING
122,340.897228,8.207162,34.158039,0.776342,0.926267,1.069117,0.822427,0.779678,0.373051,0.825708,...,0.073545,0.066999,0.079946,0.233461,0.266846,0.299092,0.226930,0.258298,0.288209,SITTING
123,17.727040,1.878521,1.776256,0.183463,0.192260,0.199627,0.188234,0.188654,0.021505,0.188293,...,0.004684,0.015061,0.017969,0.049603,0.056622,0.063587,0.048048,0.054583,0.060996,LAYING
124,430.187347,9.028684,43.104945,0.732451,1.062091,1.624140,0.904258,0.848261,1.119199,0.927564,...,0.206619,0.066492,0.082922,0.708120,0.761477,0.778275,0.706229,0.758569,0.773845,WALKING_UPSTAIRS


,0_Absolute energy,0_Area under the curve,0_Average power,0_ECDF Percentile_0,0_ECDF Percentile_1,0_Max,0_Mean,0_Median,0_Peak to peak distance,0_Root mean square,...,0_Standard deviation,0_Wavelet absolute mean_7,0_Wavelet absolute mean_8,0_Wavelet energy_6,0_Wavelet energy_7,0_Wavelet energy_8,0_Wavelet standard deviation_6,0_Wavelet standard deviation_7,0_Wavelet standard deviation_8,Activity
0,15.877739,1.269697,1.590956,0.073849,0.115618,0.840807,0.116540,0.091932,1.193411,0.178201,...,0.134810,0.028233,0.033021,0.198406,0.222017,0.249537,0.197005,0.220215,0.247343,LAYING
1,6.821886,1.160060,0.683556,0.107375,0.124901,0.169396,0.116288,0.115496,0.083123,0.116807,...,0.010994,0.009816,0.011631,0.035688,0.039755,0.043774,0.034756,0.038525,0.042200,LAYING
2,4.153816,0.821470,0.416214,0.046852,0.118585,0.252132,0.082228,0.068772,0.241005,0.091146,...,0.039321,0.005818,0.007109,0.030824,0.032070,0.035189,0.030460,0.031538,0.034463,LAYING
3,0.850436,0.409766,0.085214,-0.044469,-0.037811,-0.029742,-0.041065,-0.041150,0.021440,0.041242,...,0.003812,0.003139,0.003735,0.010845,0.012168,0.013521,0.010534,0.011756,0.012995,LAYING
4,412.251192,9.016672,41.307735,0.879704,0.922294,1.560766,0.904129,0.899124,0.953330,0.908021,...,0.083986,0.071202,0.085061,0.269501,0.307157,0.339420,0.263170,0.298790,0.328589,SITTING
5,444.294787,9.407062,44.518516,0.933025,0.951843,0.979106,0.942571,0.943104,0.080884,0.942650,...,0.012218,0.073620,0.087918,0.237663,0.271994,0.306509,0.229911,0.261841,0.293630,SITTING
6,471.512038,9.691130,47.245695,0.962886,0.979063,1.010948,0.971039,0.970738,0.076751,0.971094,...,0.010386,0.076115,0.090869,0.246947,0.282361,0.317724,0.238969,0.271908,0.304452,SITTING
7,245.836137,6.997621,24.632879,0.697045,0.705353,0.724764,0.701173,0.701127,0.037039,0.701193,...,0.005317,0.054997,0.065618,0.179579,0.205128,0.230764,0.173844,0.197618,0.221238,SITTING
8,458.788306,9.558291,45.970772,0.948995,0.967920,1.025856,0.957765,0.958687,0.148498,0.957902,...,0.016222,0.075423,0.089998,0.246451,0.281772,0.317003,0.238595,0.271491,0.303959,STANDING
9,448.472751,9.450841,44.937149,0.937170,0.956961,0.981736,0.946993,0.946969,0.087129,0.947072,...,0.012225,0.074818,0.089254,0.245764,0.280771,0.315532,0.238011,0.270620,0.302645,STANDING


3. Use the Few-Shot prompting method using UCI-HAR dataset to predict the activities that you performed. Ensure that both your examples and test query undergo similar preprocessing. How did the model perform? **[1 marks]**

In [11]:
train_list = [x_train_reduced[x_train_reduced['Activity']==act].reset_index(drop=True) for act in activity_dic.values()]
test_list = [x_test_reduced[x_test_reduced['Activity'] == act].reset_index(drop=True) for act in activity_dic.values()]

In [72]:
# System Prompts
n_test = 2 # Number of test samples per activity
n_train = 4 # Number of train samples per activity
seed = 42
predicted_activities = []
train_df = pd.concat([train_list[i].sample(n_train, random_state=seed) for i in range(6)], axis=0).reset_index(drop=True)
test_df = pd.concat([test_list[i].sample(n_test, random_state=seed) for i in range(6)], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
for test in range(n_test*6):
    query = f"""
    * You are a Human Activity Recognition model.
    * Your task is to analyze the data given in the train data provided and guess the activity being performed by the person.
    * The activities are: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
    * Provide a single label for the activity being performed by the person.
    * Feature names: {train_df.columns[:-1].to_list()}
    """
    for i in range(len(train_df)):
        query += f"* Train sample {i+1}: {train_df.iloc[i, :-1].values}\n"
        query += f"* Train label {i+1}: {train_df.iloc[i, -1]}\n"
    query += "Now, predict the activity for the following test sample:\n"
    query += f"Feature names: {train_df.columns[:-1].to_list()}\n"
    query += f"* Test sample: {test_df.iloc[test, :-1].values}\n"
    query += "Remember, just output a single label for the activity being performed. DO NOT say anything else."
    #To use Groq LLMs 
    model_name = "llama3.1-70b" # We can choose any model from the groq_models dictionary
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)
    print(f"Predicted: {answer.content}, Actual: {test_df.iloc[test, -1]}")
    predicted_activities.append(answer.content)
    time.sleep(2)
    #predicted_activities = ast.literal_eval(answer.content)
    #actual_activities = test_df["Activity"].to_list()
    #print(accuracy_score(actual_activities, predicted_activities))
    #print(answer.content)

Predicted: LAYING, Actual: LAYING
Predicted: SITTING, Actual: STANDING
Predicted: WALKING, Actual: WALKING
Predicted: STANDING, Actual: STANDING
Predicted: SITTING, Actual: WALKING_DOWNSTAIRS
Predicted: WALKING_UPSTAIRS, Actual: WALKING_UPSTAIRS
Predicted: WALKING, Actual: WALKING
Predicted: LAYING, Actual: LAYING
Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS
Predicted: SITTING, Actual: SITTING
Predicted: WALKING, Actual: WALKING_UPSTAIRS
Predicted: SITTING, Actual: SITTING


In [73]:
(np.array(predicted_activities)==np.array(test_df["Activity"].to_list())).mean()

0.75

In [76]:
# System Prompts
n_test = 4 # Number of test samples per activity
n_train = 4 # Number of train samples per activity
seed = 42
new_predicted_activities = []
new_train_df = pd.concat([train_list[i].sample(n_train, random_state=seed) for i in range(6)], axis=0).reset_index(drop=True)
new_test_df = pd.concat([test_list[i].sample(n_test, random_state=seed) for i in range(6)], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
for test in range(n_test*6):
    query = f"""
    * You are a Human Activity Recognition model.
    * Your task is to analyze the data given in the train data provided and guess the activity being performed by the person.
    * The activities are: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
    * Provide a single label for the activity being performed by the person.
    * Feature names: {new_train_df.columns[:-1].to_list()}
    """
    for i in range(len(new_train_df)):
        query += f"* Train sample {i+1}: {new_train_df.iloc[i, :-1].values}\n"
        query += f"* Train label {i+1}: {new_train_df.iloc[i, -1]}\n"
    query += "Now, predict the activity for the following test sample:\n"
    query += f"Feature names: {new_train_df.columns[:-1].to_list()}\n"
    query += f"* Test sample: {new_test_df.iloc[test, :-1].values}\n"
    query += "Remember, just output a single label for the activity being performed. DO NOT say anything else."
    #To use Groq LLMs 
    model_name = "llama3.1-70b" # We can choose any model from the groq_models dictionary
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)
    print(f"Predicted: {answer.content}, Actual: {new_test_df.iloc[test, -1]}")
    new_predicted_activities.append(answer.content)
    time.sleep(3)
    #predicted_activities = ast.literal_eval(answer.content)
    #actual_activities = test_df["Activity"].to_list()
    #print(accuracy_score(actual_activities, predicted_activities))
    #print(answer.content)

Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS
Predicted: STANDING, Actual: STANDING
Predicted: WALKING, Actual: WALKING
Predicted: STANDING, Actual: STANDING
Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS
Predicted: SITTING, Actual: WALKING_DOWNSTAIRS
Predicted: SITTING, Actual: SITTING
Predicted: WALKING, Actual: WALKING
Predicted: LAYING, Actual: LAYING
Predicted: WALKING, Actual: WALKING_UPSTAIRS
Predicted: WALKING, Actual: WALKING
Predicted: SITTING, Actual: SITTING
Predicted: STANDING, Actual: SITTING
Predicted: WALKING, Actual: WALKING
Predicted: WALKING_UPSTAIRS, Actual: WALKING_UPSTAIRS
Predicted: LAYING, Actual: LAYING
Predicted: SITTING, Actual: STANDING
Predicted: LAYING, Actual: LAYING
Predicted: LAYING, Actual: LAYING
Predicted: WALKING, Actual: WALKING_UPSTAIRS
Predicted: WALKING, Actual: WALKING_DOWNSTAIRS
Predicted: WALKING, Actual: SITTING
Predicted: STANDING, Actual: STANDING
Predicted: WALKING, Actual: WALKING_UPSTAIRS


In [78]:
(np.array(new_predicted_activities)==np.array(new_test_df["Activity"].to_list())).mean()

0.6666666666666666

In [81]:
y_test

array(['LAYING', 'LAYING', 'LAYING', 'LAYING', 'SITTING', 'SITTING',
       'SITTING', 'SITTING', 'STANDING', 'STANDING', 'STANDING',
       'STANDING', 'WALKING', 'WALKING', 'WALKING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_DOWNSTAIRS', 'WALKING_DOWNSTAIRS',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS', 'WALKING_UPSTAIRS',
       'WALKING_UPSTAIRS', 'WALKING_UPSTAIRS'], dtype='<U18')

4. Use the Few-Shot prompting method using the data you collected to predict the activities that you performed. Adopt proper processing methods as needed. How did the model perform? **[1 marks]**

In [30]:
# System Prompts
def infer(train_list, test_list,n_test, n_train, random_state=42, shuffle=False, delay=3):
    # n_test = 2 # Number of test samples per activity
    # n_train = 4 # Number of train samples per activity
    predicted_activities = []
    seed = random_state
    if shuffle:
        train_df = pd.concat([train_list[i].sample(n_train, random_state=seed) for i in range(6)], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
    else:
        train_df = pd.concat([train_list[i].sample(n_train, random_state=seed) for i in range(6)], axis=0).reset_index(drop=True)
    test_df = pd.concat([test_list[i].sample(n_test, random_state=seed) for i in range(6)], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
    for test in range(n_test*6):
        query = f"""
        * You are a Human Activity Recognition model.
        * Your task is to analyze the data given in the train data provided and guess the activity being performed by the person.
        * The activities are: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
        * Provide a single label for the activity being performed by the person.
        * Feature names: {train_df.columns[:-1].to_list()}
        """
        for i in range(len(train_df)):
            query += f"* Train sample {i+1}: {train_df.iloc[i, :-1].values}\n"
            query += f"* Train label {i+1}: {train_df.iloc[i, -1]}\n\n"
        query += "Now, predict the activity for the following test sample:\n"
        query += f"Feature names: {train_df.columns[:-1].to_list()}\n"
        query += f"* Test sample: {test_df.iloc[test, :-1].values}\n"
        query += "Remember, just output a single label for the activity being performed. DO NOT say anything else."
        #To use Groq LLMs 
        model_name = "llama3.1-70b" # We can choose any model from the groq_models dictionary
        llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
        answer = llm.invoke(query)
        print(f"Test {test}: Predicted: {answer.content}, Actual: {test_df.iloc[test, -1]}")
        predicted_activities.append(answer.content)
        time.sleep(delay)
    acc=  accuracy_score(test_df["Activity"].to_list(), predicted_activities)
    return acc, predicted_activities

In [12]:
# Leaving Hari as the test subject,
total_list = [x_test_reduced[x_test_reduced['Activity']==act].reset_index(drop=True) for act in activity_dic.values()]
train_list = [total_list[i].loc[[0, 1, 2]] for i in range(6)]
test_list = [total_list[i].loc[[3]] for i in range(6)]

In [18]:
acc, predicted_activities = infer(train_list, test_list, 1, 3, random_state=42, shuffle=False, delay=3)
print(acc)

Test 0: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING
Test 1: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_UPSTAIRS
Test 2: Predicted: LAYING, Actual: LAYING
Test 3: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS
Test 4: Predicted: STANDING, Actual: STANDING
Test 5: Predicted: WALKING, Actual: SITTING
0.5


In [13]:
# Leaving Aarsh as the test subject,
total_list = [x_test_reduced[x_test_reduced['Activity']==act].reset_index(drop=True) for act in activity_dic.values()]
train_list = [total_list[i].loc[[1, 2, 3]] for i in range(6)]
test_list = [total_list[i].loc[[0]] for i in range(6)]

In [21]:
acc, predicted_activities = infer(train_list, test_list, 1, 3, random_state=42, shuffle=False, delay=3)
print(acc)

Test 0: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING
Test 1: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_UPSTAIRS
Test 2: Predicted: LAYING, Actual: LAYING
Test 3: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS
Test 4: Predicted: STANDING, Actual: STANDING
Test 5: Predicted: WALKING, Actual: SITTING
0.5


In [22]:
# Leaving Abhinav as the test subject:
total_list = [x_test_reduced[x_test_reduced['Activity']==act].reset_index(drop=True) for act in activity_dic.values()]
train_list = [total_list[i].loc[[0, 2, 3]] for i in range(6)]
test_list = [total_list[i].loc[[1]] for i in range(6)]
acc, predicted_activities = infer(train_list, test_list, 1, 3, random_state=42, shuffle=False, delay=3)
print(acc)

Test 0: Predicted: WALKING, Actual: WALKING
Test 1: Predicted: WALKING, Actual: WALKING_UPSTAIRS
Test 2: Predicted: LAYING, Actual: LAYING
Test 3: Predicted: WALKING, Actual: WALKING_DOWNSTAIRS
Test 4: Predicted: STANDING, Actual: STANDING
Test 5: Predicted: STANDING, Actual: SITTING
0.5


In [31]:
# Leaving Arjun as the test subject:
total_list = [x_test_reduced[x_test_reduced['Activity']==act].reset_index(drop=True) for act in activity_dic.values()]
train_list = [total_list[i].loc[[0, 1, 3]] for i in range(6)]
test_list = [total_list[i].loc[[2]] for i in range(6)]
acc, predicted_activities = infer(train_list, test_list, 1, 3, random_state=42, shuffle=False, delay=3)
print(acc)


Test 0: Predicted: WALKING, Actual: WALKING
Test 1: Predicted: WALKING, Actual: WALKING_UPSTAIRS
Test 2: Predicted: LAYING, Actual: LAYING
Test 3: Predicted: WALKING_UPSTAIRS, Actual: WALKING_DOWNSTAIRS
Test 4: Predicted: STANDING, Actual: STANDING
Test 5: Predicted: STANDING, Actual: SITTING
0.5


It can therefore be seen that training the model only on our collected data yielded a accuracy of 0.5 for all test subjects. We passed in the whole training data (18 samples) and predicted the labels of the test data, which was made up of the 6 samples from the subject left out.